### Library

In [79]:
%pip install requests
%pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [80]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

# import textblob
from textblob import Word
from textblob import TextBlob

In [81]:
r = requests.get("https://www.yelp.com/biz/kia-of-marin-novato")
soup = BeautifulSoup(r.text, "html.parser")

In [82]:
r.status_code

200

## Pulling data from the internet using BeautifulSoup

In [83]:
def create_data():
    comments =  []
    userName = []
    rating = []
    
    
    soup_lis = soup \
    .find('ul', {'class': 'list__09f24__ynIEd'})\
    .find_next('li', {'class': 'y-css-1jp2syp'})\
    
    
    #Yorumlar kısmını çekme
    for lis in soup_lis:
        li = lis.find_next("div", {'class': 'y-css-1iy1dwt'})\
            .find_next("div", {'class': 'y-css-cluvhg'})\
            .find_all_next("p", {'class': 'comment__09f24__D0cxf y-css-h9c2fl'})
            
        for l in li:
            comments.append(l.text)
    
    #Kullanici adlarını çekme    
    for lis in soup_lis:
        li = lis.find_next("div", {'class': 'y-css-1iy1dwt'})\
            .find_next("span", {'class': 'y-css-w3ea6v'})\
            .find_all_next("a", {"class": "y-css-12ly5yx"})
            
        for l in li:
            if l.text != "1 photo":
                userName.append(l.text)

    #Kaç yıldız verdiklerini görürüz. Onları çekeriz.
    for lis in soup_lis:
        li = lis.find_next("div", {'class': 'y-css-1iy1dwt'})\
            .find_next("div", {'class': 'y-css-19pbem2'})\
            .find_next("span", {'class': 'y-css-pw0opj'})\
            .find_all_next("div", attrs={"class": 'y-css-9tnml4'})
            
            
        for l in li:
            #print(l['aria-label'])
            rating.append(l['aria-label'])
        
        rating = rating[:10]

    return comments, userName, rating

In [84]:
comments, userName, rating = create_data()

## Analysis The Data

In [85]:
data = pd.DataFrame(userName, columns=["UserName"])
data["Rating"] = pd.DataFrame(rating, columns=["Rating"])
data["Review"] = pd.DataFrame(comments, columns=["Review"])

In [86]:
data.head()

,UserName,Rating,Review
0,Maureen P.,5 star rating,Even an EV needs service from time to time. Ki...
1,Isis L.,5 star rating,I wasn't looking to buy a car as I was content...
2,Patrick H.,5 star rating,Worked with Julio and he was excellent from st...
3,Koichi D.,5 star rating,Been dealing with Kia Marin for years. Leased ...
4,Janessa B.,5 star rating,Been bringing my Kia k5 here for a couple of y...


In [87]:
len(data['Review'])

10

In [88]:
data_word = pd.DataFrame(data["Review"]) 

In [89]:
data_word

,Review
0,Even an EV needs service from time to time. Ki...
1,I wasn't looking to buy a car as I was content...
2,Worked with Julio and he was excellent from st...
3,Been dealing with Kia Marin for years. Leased ...
4,Been bringing my Kia k5 here for a couple of y...
5,Unprofessional sales team. They dismiss conver...
6,I bought a new Kia Sorento Hybrid from Kia of ...
7,I recently purchased a used car from this deal...
8,Best car buying experience. Julio was awesom...
9,I usually only post reviews on Yelp when I hav...


In [90]:
data_word["Word_count"] = data_word['Review'].apply(lambda x: len(x.split()))

In [91]:
data_word["Char_count"] = data_word['Review'].apply(lambda x: len(x))

In [92]:
import nltk
from nltk.corpus import stopwords

In [93]:
stop_words = stopwords.words("english")

In [94]:
len(stop_words)

179

In [95]:
data_word["stopword_counter"] = data_word["Review"].apply(lambda x: len([word for word in x.split() if word.lower() in stop_words]))

In [96]:
data_word["stopword_rate"] = data_word["stopword_counter"] / data_word["Word_count"]

In [97]:
data.head()

,UserName,Rating,Review
0,Maureen P.,5 star rating,Even an EV needs service from time to time. Ki...
1,Isis L.,5 star rating,I wasn't looking to buy a car as I was content...
2,Patrick H.,5 star rating,Worked with Julio and he was excellent from st...
3,Koichi D.,5 star rating,Been dealing with Kia Marin for years. Leased ...
4,Janessa B.,5 star rating,Been bringing my Kia k5 here for a couple of y...


In [98]:
data_word

,Review,Word_count,Char_count,stopword_counter,stopword_rate
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484
5,Unprofessional sales team. They dismiss conver...,53,355,22,0.415094
6,I bought a new Kia Sorento Hybrid from Kia of ...,110,564,57,0.518182
7,I recently purchased a used car from this deal...,136,758,65,0.477941
8,Best car buying experience. Julio was awesom...,39,258,13,0.333333
9,I usually only post reviews on Yelp when I hav...,186,998,88,0.473118


In [99]:
data_word.sort_values(by = "stopword_rate")

,Review,Word_count,Char_count,stopword_counter,stopword_rate
8,Best car buying experience. Julio was awesom...,39,258,13,0.333333
5,Unprofessional sales team. They dismiss conver...,53,355,22,0.415094
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622
9,I usually only post reviews on Yelp when I hav...,186,998,88,0.473118
7,I recently purchased a used car from this deal...,136,758,65,0.477941
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484
6,I bought a new Kia Sorento Hybrid from Kia of ...,110,564,57,0.518182


In [100]:
data_word.describe()

,Word_count,Char_count,stopword_counter,stopword_rate
count,10.000000,10.000000,10.000000,10.000000
mean,130.700000,702.900000,61.200000,0.457159
std,89.410477,422.602111,42.855831,0.055486
min,39.000000,258.000000,13.000000,0.333333
25%,85.250000,473.000000,38.000000,0.431894
50%,100.500000,537.500000,52.000000,0.472870
75%,163.000000,900.500000,73.250000,0.490272
max,347.000000,1691.000000,164.000000,0.518182


## Data Cleaning

In [101]:
data_word["lowercase"] = data_word["Review"].apply(lambda x: " ".join(word.lower() for word in x.split()))
data_word["punction"] = data_word["lowercase"].str.replace("[^\w\s]", "")
data_word["stopword"] = data_word["punction"].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

In [102]:
data_word.head()

,Review,Word_count,Char_count,stopword_counter,stopword_rate,lowercase,punction,stopword
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860,even an ev needs service from time to time. ki...,even an ev needs service from time to time. ki...,even ev needs service time time. kia marin hit...
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622,i wasn't looking to buy a car as i was content...,i wasn't looking to buy a car as i was content...,looking buy car content ny 2007 toyota yaris a...
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382,worked with julio and he was excellent from st...,worked with julio and he was excellent from st...,worked julio excellent start finish. knowledge...
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571,been dealing with kia marin for years. leased ...,been dealing with kia marin for years. leased ...,dealing kia marin years. leased many kia's yea...
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484,been bringing my kia k5 here for a couple of y...,been bringing my kia k5 here for a couple of y...,"bringing kia k5 couple years must say, service..."


In [103]:
pd.Series("".join(data_word["stopword"]).split()).value_counts()[:20]

kia           15
car           14
service        8
experience     6
time           6
key            6
hugh           5
one            5
used           4
call           4
made           4
many           4
going          4
new            4
ev             4
within         3
buying         3
car.           3
got            3
help           3
Name: count, dtype: int64

## Lemmatization

In [104]:
data_word["lemmatize"] = data_word["stopword"].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))

## Sentiment Analysis

In [105]:
data_word["polarity"] = data_word["lemmatize"].apply(lambda x: TextBlob(x).sentiment[0])
data_word["subjectivity"] = data_word["lemmatize"].apply(lambda x: TextBlob(x).sentiment[1])

In [106]:
data_word.head()

,Review,Word_count,Char_count,stopword_counter,stopword_rate,lowercase,punction,stopword,lemmatize,polarity,subjectivity
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860,even an ev needs service from time to time. ki...,even an ev needs service from time to time. ki...,even ev needs service time time. kia marin hit...,even ev need service time time. kia marin hit ...,0.394444,0.669306
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622,i wasn't looking to buy a car as i was content...,i wasn't looking to buy a car as i was content...,looking buy car content ny 2007 toyota yaris a...,looking buy car content ny 2007 toyota yaris a...,0.318008,0.611127
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382,worked with julio and he was excellent from st...,worked with julio and he was excellent from st...,worked julio excellent start finish. knowledge...,worked julio excellent start finish. knowledge...,0.475000,0.665625
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571,been dealing with kia marin for years. leased ...,been dealing with kia marin for years. leased ...,dealing kia marin years. leased many kia's yea...,dealing kia marin years. leased many kia's yea...,0.307589,0.606200
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484,been bringing my kia k5 here for a couple of y...,been bringing my kia k5 here for a couple of y...,"bringing kia k5 couple years must say, service...","bringing kia k5 couple year must say, service ...",0.313542,0.515625


In [107]:
data_word.columns

Index(['Review', 'Word_count', 'Char_count', 'stopword_counter',
       'stopword_rate', 'lowercase', 'punction', 'stopword', 'lemmatize',
       'polarity', 'subjectivity'],
      dtype='object')

In [108]:
data_word.drop(['lowercase', 'punction', 'stopword', 'lemmatize'], axis=1, inplace=True)

In [109]:
data_word.head()

,Review,Word_count,Char_count,stopword_counter,stopword_rate,polarity,subjectivity
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860,0.394444,0.669306
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622,0.318008,0.611127
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382,0.475000,0.665625
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571,0.307589,0.606200
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484,0.313542,0.515625


In [113]:
data_word.sort_values(by="polarity")

,Review,Word_count,Char_count,stopword_counter,stopword_rate,polarity,subjectivity
9,I usually only post reviews on Yelp when I hav...,186,998,88,0.473118,0.008117,0.539881
5,Unprofessional sales team. They dismiss conver...,53,355,22,0.415094,0.016667,0.483333
7,I recently purchased a used car from this deal...,136,758,65,0.477941,0.135714,0.602778
6,I bought a new Kia Sorento Hybrid from Kia of ...,110,564,57,0.518182,0.275118,0.535458
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571,0.307589,0.606200
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484,0.313542,0.515625
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622,0.318008,0.611127
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860,0.394444,0.669306
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382,0.475000,0.665625
8,Best car buying experience. Julio was awesom...,39,258,13,0.333333,0.539167,0.669167
